# Combining and analysing the low density anglescan simulations for all 5 probe tips

In [21]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import xarray as xr
import pandas as pd
import scipy.io as sio
import sys
import os
import glob
import copy
import pathlib as pth
import importlib
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.spice.splopter as spl
import flopter.spice.tdata as td
import flopter.spice.homogenise as hmg
import flopter.core.ivdata as iv
import flopter.core.fitters as fts
import flopter.core.fitdata as fd
import flopter.core.lputils as lpu

In [20]:
importlib.reload(spl)
importlib.reload(td)
importlib.reload(hmg)
importlib.reload(fd)
importlib.reload(lpu)

<module 'flopter.core.lputils' from '/home/jleland/Coding/Projects/flopter/flopter/core/lputils.py'>

## Paths and Matfile IO
This section deals with file io and selecting the right .mat files. This needs to be run for the latter sections to work.

In [73]:
spice_dir = pth.Path('/home/jleland/Data/external/spice/')
os.chdir(spice_dir)

In [74]:
lps = lpu.MagnumProbes()

flush_probe = copy.deepcopy(lps.probe_l)
flush_probe.theta_p = 0.0
flush_probe.d_perp = 0.0

angled_probe = copy.deepcopy(lps.probe_l)
angled_probe.d_perp = 0.0
print(angled_probe.theta_p)

0.17453292519943295


In [79]:
skippable_scans = {
    'marconi/lowdens_anglescan/inpartest_as',
    'marconi/lowdens_anglescan/angled_tiltscan_ext'
}
single_sims = {
    'marconi/restart_test/sprobe_9hr_multirestart',
    'marconi/restart_test/sprobe_control',
    'marconi/restart_test/sprobe_1hr',
    'marconi/restart_test/sprobe_1hr.1',
    'marconi/restart_test/sprobe_1hr_atrestart',
    'marconi/restart_test/sprobe_4hr_multirestart_r',
    'marconi/restart_test/sprobe_4hr_multirestart_2r'
}

In [80]:
scans_searchstr = '*/restart_test/*'
# angles_search_str = '/*[!.{yml, inp}]/backup*'
angles_search_str = '/*[!.{yml, inp}]'

non_standard_variables = {'ProbePot', 'npartproc', 'Nz', 'Nzmax', 'Ny', 'count', 'Npc', 'snumber', 'nproc'}

desired_variables = td.DEFAULT_REDUCED_DATASET | non_standard_variables

all_run_dirs = {}
scans = glob.glob(scans_searchstr)
scans = set(scans) - skippable_scans
for scan in scans:
    if scan in single_sims:
        print(f'Single sim {scan} found')
        all_run_dirs[scan] = [scan]
    else:
        all_run_dirs[scan] = glob.glob(scan + angles_search_str)
    
    print(f'{scan}: {all_run_dirs[scan]}\n')


Single sim marconi/restart_test/sprobe_4hr_multirestart_2r found
marconi/restart_test/sprobe_4hr_multirestart_2r: ['marconi/restart_test/sprobe_4hr_multirestart_2r']

Single sim marconi/restart_test/sprobe_9hr_multirestart found
marconi/restart_test/sprobe_9hr_multirestart: ['marconi/restart_test/sprobe_9hr_multirestart']

Single sim marconi/restart_test/sprobe_1hr_atrestart found
marconi/restart_test/sprobe_1hr_atrestart: ['marconi/restart_test/sprobe_1hr_atrestart']

Single sim marconi/restart_test/sprobe_control found
marconi/restart_test/sprobe_control: ['marconi/restart_test/sprobe_control']

Single sim marconi/restart_test/sprobe_1hr.1 found
marconi/restart_test/sprobe_1hr.1: ['marconi/restart_test/sprobe_1hr.1']

Single sim marconi/restart_test/sprobe_4hr_multirestart_r found
marconi/restart_test/sprobe_4hr_multirestart_r: ['marconi/restart_test/sprobe_4hr_multirestart_r']

Single sim marconi/restart_test/sprobe_1hr found
marconi/restart_test/sprobe_1hr: ['marconi/restart_test/s

## Plotting both IV characterisitics on the same axes

In [81]:
skippable = {}
scans = list(scans)

In [82]:
# fig, axes = plt.subplots(2)
splopters = {}

for i in range(len(scans)):
#     ax = axes[i]
    scan = scans[i]

    for angle_dir in all_run_dirs[scan]:
        print(f'\nCreating splopter for {scan}, {angle_dir}')
        if angle_dir in skippable:
            print('Skipping...')
            continue
        try:
            splopter = spl.Splopter(spice_dir / angle_dir, reduce=desired_variables)
            splopter.prepare(denormaliser_fl=True, homogenise_fl=True, find_se_temp_fl=False)
    #         splopter.denormalise()
            splopters[angle_dir] = splopter
        except AssertionError as e:
            print(f'AssertionError encountered for {angle_dir}. Skipping...')
            continue
        
print('\n...Done')


Creating splopter for marconi/restart_test/sprobe_4hr_multirestart_2r, marconi/restart_test/sprobe_4hr_multirestart_2r
No a-file given, continuing without

Creating splopter for marconi/restart_test/sprobe_9hr_multirestart, marconi/restart_test/sprobe_9hr_multirestart
Looking for a suitable backup to use instead.
Useable backup found at /home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0032/t-sprobe_9hr_multirestart.mat

Creating splopter for marconi/restart_test/sprobe_1hr_atrestart, marconi/restart_test/sprobe_1hr_atrestart
No a-file given, continuing without

Creating splopter for marconi/restart_test/sprobe_control, marconi/restart_test/sprobe_control

Creating splopter for marconi/restart_test/sprobe_1hr.1, marconi/restart_test/sprobe_1hr.1
No a-file given, continuing without

Creating splopter for marconi/restart_test/sprobe_4hr_multirestart_r, marconi/restart_test/sprobe_4hr_multirestart_r
No a-file given, continuing without

Creatin

/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/home/jleland/Coding/Environments/python3.6/lib/python3.6/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


No a-file given, continuing without

...Done


In [83]:
for i, (filename, splopter) in enumerate(splopters.items()):
    print(f'{i}) {filename}: {splopter}')

0) marconi/restart_test/sprobe_4hr_multirestart_2r: <flopter.spice.splopter.Splopter object at 0x7f6cb30560f0>
1) marconi/restart_test/sprobe_9hr_multirestart: <flopter.spice.splopter.Splopter object at 0x7f6cb27275f8>
2) marconi/restart_test/sprobe_1hr_atrestart: <flopter.spice.splopter.Splopter object at 0x7f6cb2751da0>
3) marconi/restart_test/sprobe_control: <flopter.spice.splopter.Splopter object at 0x7f6cb27cf4e0>
4) marconi/restart_test/sprobe_1hr.1: <flopter.spice.splopter.Splopter object at 0x7f6cb277e550>
5) marconi/restart_test/sprobe_4hr_multirestart_r: <flopter.spice.splopter.Splopter object at 0x7f6cb2852940>
6) marconi/restart_test/sprobe_1hr: <flopter.spice.splopter.Splopter object at 0x7f6cb2889b38>


In [11]:
# For adding a new run to the list of splopters
angle_dir = all_run_dirs[scans[6]][0]
print(angle_dir)

if angle_dir in splopters:
    print(f'Removing {angle_dir} from existing list of splopters')
    splopters.pop(angle_dir)
splopter = spl.Splopter(spice_dir / angle_dir, reduce=desired_variables)
splopter.prepare(denormaliser_fl=True, homogenise_fl=True, find_se_temp_fl=False)
splopter.denormalise()
splopters[angle_dir] = splopter

IndexError: list index out of range

## Fitting and Adding Info to a Dataframe
Might be good to make it into a dataset instead so individual IV chracterisitcs can be stored as well.

In [85]:
fig, axes = plt.subplots(3)

fit_df = pd.DataFrame(columns=['probe', 
                               'angle', 
                               'temp',
                               'd_temp',
                               'isat', 
                               'd_isat',
                               'a',
                               'd_a',
                               'v_f',
                               'd_v_f'
                              ])

for label, s in splopters.items():
    print('=== ' + label + ' ===')
    
    label = label.split('/')[-1]
    
    probe = lps.probe_s
    ax = axes[0]
    axt = axes[1]
    axd = axes[2]
    
    angle = np.degrees(np.abs(np.squeeze(s.tdata.alphayz)))
    
    if any(len(data_arr) == 0 for variable, data_arr in s.iv_data.items()):
        print(f'Found empty data array in {label}')
        continue
    
    iv_region = np.where(s.iv_data['V'] < -2)
    i = np.where((np.diff(s.iv_data['V'][iv_region]) == 0))
    i = (i[0] + 1, )
    
    i = iv_region
#     s.denormalise()
    
    ax.errorbar(s.iv_data['V'][i], s.iv_data['I'][i], label=angle, yerr=s.iv_data['sigma'][i])
    ax.legend()
    
    try:
        fit_iv_data = iv.IVData.non_contiguous_trim(s.iv_data, i)
        
        if any(len(data_arr) == 0 for variable, data_arr in fit_iv_data.items()):
            print(f'Found empty data array in {label}')
            continue
        fitter = fts.FullIVFitter()
        fit_data = fitter.fit_iv_data(fit_iv_data)
        #     ax.plot(*fit_data.get_fit_plottables())

        fit_df = fit_df.append({'probe': probe, 
                                'angle': angle, 
                                'temp': fit_data.get_temp(),
                                'd_temp': fit_data.get_temp_err(),
                                'isat': fit_data.get_isat(), 
                                'd_isat': fit_data.get_isat_err(),
                                'a': fit_data.get_sheath_exp(), 
                                'd_a': fit_data.get_sheath_exp_err(),
                                'v_f': fit_data.get_floating_pot(),
                                'd_v_f': fit_data.get_floating_pot_err()}, ignore_index=True)

        axt.errorbar(angle, fit_data.get_temp(), yerr=fit_data.get_temp_err(), fmt='x') #, ecolor='silver')
        axd.errorbar(angle, fit_data.get_isat(), yerr=fit_data.get_isat_err(), fmt='x') #, ecolor='silver')
    except RuntimeError as e:
        print(f'WARNING: Encountered error in fit, skipping {label}')

for j, probe in enumerate(['flush', 'angled']):
    filt_df = fit_df.where(fit_df.probe == probe).dropna()
    axt = axes[j][1]
    axd = axes[j][2]
    if j == 0:
        axt.set_title(r'$T_e$')
        axd.set_title(r'$I_{sat}$')
    if j == 1:
        axt.set_xlabel(r'$\theta$')
        axd.set_xlabel(r'$\theta$')
#     axt.errorbar(filt_df['angle'], filt_df['temp'] - 1, yerr=filt_df['d_temp'], fmt='x', ecolor='silver')
#     axt.set_ylim(0.3, 1.7)
    axt.axhline(y=1.0, color='k', linewidth=0.5, linestyle='dotted')
#     axd.errorbar(filt_df['angle'], filt_df['isat'], yerr=filt_df['d_isat'], fmt='x', ecolor='silver')
#     axd.set_ylim(-0.1, 90)
    
fit_df


=== marconi/restart_test/sprobe_4hr_multirestart_2r ===
=== marconi/restart_test/sprobe_9hr_multirestart ===


AttributeError: 'NoneType' object has no attribute 'items'

The above cell is supposed to plot the two simulations of the same input file on teh same set of axes, but the data in the multi-restart directory is not homogenisable. Will need to look into this in more detail. 

## Analysis of the backups of sprobe_9hr_multirestart

In [91]:
chosen_spl_path = spice_dir / all_run_dirs[scans[5]][0]
print(chosen_spl_path)

splopter = spl.Splopter(chosen_spl_path, reduce=desired_variables)
splopter.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False)
# universal_probe_bias = np.squeeze(splopter.tdata.diagnostics['ProbePot'])

/home/jleland/Data/external/spice/marconi/restart_test/sprobe_4hr_multirestart_r
No a-file given, continuing without


In [25]:
universal_probe_bias

array([-9.9 , -9.9 , -9.9 , ..., 10.05, 10.05, 10.05])

In [29]:
def plot_raw_data_splopter(path_to_splopter, axes=None):
    label = str(path_to_splopter).split('/')[-1]
    print(label)
    
    splopter = spl.Splopter(path_to_splopter, reduce=desired_variables)
    splopter.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False)

    # Extract relevant arrays from the matlab file
    probe_indices = splopter.parser.get_probe_obj_indices()
    probe_current_e = 0.0
    probe_current_i = 0.0

    time = np.squeeze(splopter.tdata.t)[:-1]
    for index in probe_indices:
        probe_current_e += np.squeeze(splopter.tdata.objectscurrente)[index]
        probe_current_i += np.squeeze(splopter.tdata.objectscurrenti)[index]
    probe_bias = np.squeeze(splopter.tdata.diagnostics['ProbePot'])
    probe_current_tot = probe_current_i + probe_current_e
    
    print(splopter.tdata.diagnostics)
    
    if axes is None:
        fig, axes = plt.subplots(2)
    
    axes[0].set_title(label)
    axes[0].plot(probe_bias, label=label)
#     axes[0].plot(universal_probe_bias)
    axes[0].legend()
    axes[1].plot(probe_current_tot, label=label)
#     axes[1].plot(time, probe_current_i)
#     axes[1].plot(time, probe_current_e)
    axes[1].legend()

plot_raw_data_splopter(chosen_spl_path)

sprobe_control
{'ProbePot': array([[-9.9 ],
       [-9.9 ],
       [-9.9 ],
       ...,
       [10.05],
       [10.05],
       [10.05]])}


/home/jleland/.pyenv/versions/3.6.5/lib/python3.6/tkinter/__init__.py:746: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  func(*args)
/home/jleland/.pyenv/versions/3.6.5/lib/python3.6/tkinter/__init__.py:1702: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.
  return self.func(*args)


In [97]:
backups = list(chosen_spl_path.glob('backup*'))
backups.sort()

bu_splopters = []
for i, bu_folder in enumerate(backups):
    print(f'{i}) {str(bu_folder)}')
    
    splopter = spl.Splopter(bu_folder, reduce=desired_variables)
    splopter.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False)
    bu_splopters.append(splopter)
    

In [106]:
print(plot_concatted_current(bu_splopters))

None


In [99]:
fig, axes = plt.subplots(2)
for bu_folder in backups:
    plot_raw_data_splopter(bu_folder)

backup_20191018-1718
No a-file given, continuing without
{'ProbePot': array([[0.]])}
backup_20191018-2255
No a-file given, continuing without
{'ProbePot': array([[0.]])}
backup_20191019-0005
No a-file given, continuing without
{'ProbePot': array([[-9.9],
       [-9.9],
       [-9.9],
       ...,
       [-9.9],
       [-9.9],
       [-9.9]])}
backup_20191019-0116
No a-file given, continuing without
{'ProbePot': array([[-9.9],
       [-9.9],
       [-9.9],
       ...,
       [-9.9],
       [-9.9],
       [-9.9]])}


## Comparing manually restarted simulations

In [86]:
fig, axes = plt.subplots(2)
for path, s in splopters.items():
    if '4hr' not in str(path):
        continue
    plot_raw_data_splopter(path, axes=axes)

sprobe_4hr_multirestart_2r
No a-file given, continuing without
{'ProbePot': array([[-9.9],
       [-9.9],
       [-9.9],
       ...,
       [-9.9],
       [-9.9],
       [-9.9]])}
sprobe_4hr_multirestart_r
No a-file given, continuing without
{'ProbePot': array([[-9.9],
       [-9.9],
       [-9.9],
       ...,
       [-9.9],
       [-9.9],
       [-9.9]])}


In [134]:
def plot_concatted_current(splopters, trim_concat=False, ax=None):
    concatted_time = np.array([])
    concatted_current = np.array([])
    if trim_concat:
        lengths = []
        
    if ax is None:
        fig, ax = plt.subplots()
    
    for s in splopters:

        # Extract relevant arrays from the matlab file
        probe_indices = s.parser.get_probe_obj_indices()
        probe_current_e = 0.0
        probe_current_i = 0.0

        time = np.squeeze(s.tdata.t)[:-1]
        for index in probe_indices:
            probe_current_e += np.squeeze(s.tdata.objectscurrente)[index]
            probe_current_i += np.squeeze(s.tdata.objectscurrenti)[index]
        probe_bias = np.squeeze(s.tdata.diagnostics['ProbePot'])
        probe_current_tot = probe_current_i + probe_current_e
        
        if trim_concat:
            concatcurr_len = len(concatted_current)
            concatted_current = np.concatenate([concatted_current, probe_current_tot[concatcurr_len:]])
            ax.axvline(x=concatcurr_len)
            lengths.append(concatcurr_len)
        else:
            print(f'time: {len(time)}, current: {len(probe_current_tot)}')
            concatted_current = np.concatenate([concatted_current, probe_current_tot])
            if len(concatted_time) > 0:
                offset = concatted_time[-1]
            else:
                offset = 0
            concatted_time = np.concatenate([concatted_time, offset + time])
    
    if trim_concat:
        ax.plot(concatted_current)
        return lengths
    else:
        ax.plot(concatted_time, concatted_current)

In [57]:
splopters_reordered = [
    splopters['marconi/restart_test/sprobe_1hr_atrestart'],
    splopters['marconi/restart_test/sprobe_1hr']
]

plot_concatted_current(splopters_reordered)

marconi/restart_test/sprobe_1hr_atrestart
marconi/restart_test/sprobe_1hr


In [61]:
bu_splopters = {}

for bu_folder in backups:
    splopter = spl.Splopter(bu_folder, reduce=desired_variables)
    splopter.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False)
    
    bu_splopters[bu_folder] = splopter

No a-file given, continuing without
No a-file given, continuing without
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without
No a-file given, continuing without
There are no backups to utilise. Carrying on, but expect erroneous results.
No a-file given, continuing without
There are no backups to utilise. Carrying on, but expect erroneous results.
There are no backups to utilise. Carrying on, but expect erroneous results.
There are no backups to utilise. Carrying on, but expect erroneous results.


In [68]:
for bu_s in bu_splopters.values():
    print(bu_s.data_dir)

/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0032
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0133
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0233
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0334
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0434
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0535
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0635
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0736
/home/jleland/Data/external/spice/marconi/restart_test/sprobe_9hr_multirestart/backup_20191018-0838


In [72]:
print(plot_concatted_current(bu_splopters.values()))

[0, 1296, 1296, 1769, 1769, 1769, 1769, 1800, 1800]


## Looking at -r restart sims

In [115]:
for i, scan in enumerate(scans):
    print(i, scan)

0 marconi/restart_test/sprobe_1hr
1 marconi/restart_test/sprobe_1hr.1
2 marconi/restart_test/sprobe_1hr_atrestart
3 marconi/restart_test/sprobe_4hr_multirestart_2r
4 marconi/restart_test/sprobe_4hr_multirestart_r
5 marconi/restart_test/sprobe_9hr_multirestart
6 marconi/restart_test/sprobe_control


In [123]:
scans_comparison = [scans[6], scans[4], scans[3],]
scans_comparison.sort()
print(scans_comparison)

['marconi/restart_test/sprobe_4hr_multirestart_2r', 'marconi/restart_test/sprobe_4hr_multirestart_r', 'marconi/restart_test/sprobe_control']


In [137]:
fig, axes = plt.subplots(1, sharex=True)

for i, scan in enumerate(scans_comparison):
    backups = list(pth.Path(scan).glob('backup*'))
    backups.sort()

    bu_splopters = []
    for j, bu_folder in enumerate(backups):
        print(f'{j}) {str(bu_folder)}')

        splopter = spl.Splopter(bu_folder, reduce=desired_variables)
        splopter.prepare(denormaliser_fl=False, homogenise_fl=True, find_se_temp_fl=False)
        bu_splopters.append(splopter)
    
    plot_concatted_current(bu_splopters, ax=axes)
    axes.set_title(scan.split('/')[-1])
    

0) marconi/restart_test/sprobe_4hr_multirestart_2r/backup_20191018-1818
No a-file given, continuing without
1) marconi/restart_test/sprobe_4hr_multirestart_2r/backup_20191019-0000
No a-file given, continuing without
time: 1300, current: 1300
time: 1011, current: 1011
0) marconi/restart_test/sprobe_4hr_multirestart_r/backup_20191018-1718
No a-file given, continuing without
1) marconi/restart_test/sprobe_4hr_multirestart_r/backup_20191018-2255
No a-file given, continuing without
2) marconi/restart_test/sprobe_4hr_multirestart_r/backup_20191019-0005
No a-file given, continuing without
3) marconi/restart_test/sprobe_4hr_multirestart_r/backup_20191019-0116
No a-file given, continuing without
time: 1312, current: 1312
time: 1202, current: 1202
time: 1030, current: 1030
time: 1877, current: 1877
0) marconi/restart_test/sprobe_control/backup_20191018-0758
time: 1110, current: 1110
